In [1]:
# We need to create a SQLite database
# I'm going to use a API, but you can use CSVs

# Load data into a PANDAS dataframe -> load into the database

#### Load Dependencies

In [1]:
# data science
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import datetime

# API
import requests
import json

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, text, inspect, func

#### Read in Data File

In [3]:
# Create a file path
filepath = "../Resources/scrubbed.csv"

# Read in the data. 
df = pd.read_csv(filepath)

df.head()

C:\Users\eugch\AppData\Local\Temp\ipykernel_33728\3153306518.py:5: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611


#### Data Cleaning

In [6]:
# Fill nulls with 'unknown'
df.country = df.country.fillna('unknown')
df.state = df.state.fillna('unknown')
df["shape"] = df["shape"].fillna('unknown')
df.info()

In [7]:
# Create a pared down list of shapes
item_shape = ['hexagon', 'delta', 'round', 'crescent', 'pyramid', 'flare', 'dome', 'changed', 'cross', 'cone']
df = df[~df["shape"].isin(item_shape)]

#### Below is the remaining starter code

In [4]:
# Create engine using the `ufo.sqlite` database file
engine = create_engine("sqlite:///ufo.sqlite")

In [6]:
# write to the database
df.to_sql("launchpads", con=engine, index=False, method="multi", if_exists="replace")

6

In [7]:
# INSPECT to confirm existence

# Create the inspector and connect it to the engine
inspector_gadget = inspect(engine)

# Collect the names of tables within the database
tables = inspector_gadget.get_table_names()

# print metadata for each table
for table in tables:
    print(table)
    print("-----------")
    
    # get columns
    columns = inspector_gadget.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])

    print()

launchpads
-----------
images TEXT
name TEXT
full_name TEXT
locality TEXT
region TEXT
latitude FLOAT
longitude FLOAT
launch_attempts BIGINT
launch_successes BIGINT
rockets BIGINT
timezone TEXT
status TEXT
details TEXT
id TEXT



In [8]:
# close the engine
engine.dispose()